In [19]:
data = """#.#####################
#.......#########...###
#######.#########.#.###
###.....#.>.>.###.#.###
###v#####.#v#.###.#.###
###.>...#.#.#.....#...#
###v###.#.#.#########.#
###...#.#.#.......#...#
#####.#.#.#######.#.###
#.....#.#.#.......#...#
#.#####.#.#.#########v#
#.#...#...#...###...>.#
#.#.#v#######v###.###v#
#...#.>.#...>.>.#.###.#
#####v#.#.###v#.#.###.#
#.....#...#...#.#.#...#
#.#########.###.#.#.###
#...###...#...#...#.###
###.###.#.###v#####v###
#...#...#.#.>.>.#.>.###
#.###.###.#.###.#.#v###
#.....###...###...#...#
#####################.#
"""
data = open('puzzle.data').read()

from collections import defaultdict
from helper import *

def get_directions(grid: Grid, pos: complex) -> list[complex]:
    waypoints = []
    for check_dir in FOUR_DIRECTIONS:
        if pos + check_dir not in grid or grid[pos + check_dir] != '#':
            waypoints.append(check_dir)
    return waypoints

ok_dir_map = {'.': FOUR_DIRECTIONS, '>': [RIGHT], '<': [LEFT], '^': [UP], 'v': [DOWN]}

def solve(data: str, check_slopes=True):
    grid = Grid.from_str(data)
    waypoints: dict[complex, list[tuple[complex, int]]] = defaultdict(list)
    
    open_list = [(1 + 0j, DOWN)]
    visited = set()
    
    while open_list:
        start, dir = open_list.pop()
        if (start, dir) in visited:
            continue
        visited.add( (start, dir) )
        pos = start + dir
        length = 1
        while True:
            if pos.imag == grid.height - 1:
                waypoints[start].append( (pos, length) )
                break
            if pos not in grid or (check_slopes and dir not in ok_dir_map[grid[pos]]):
                break
            next_dir = get_directions(grid, pos)
            if len(next_dir) == 1:
                break
            length += 1
            if len(next_dir) > 2:
                waypoints[start].append( (pos, length) )
                open_list.extend([(pos, d) for d in next_dir if d != -dir])
                break
            dir = next_dir[0] if next_dir[0] != -dir else next_dir[1]
            pos += dir

    open_list = [((1 + 0j,), 0)]
    end_paths = []
    while open_list:
        path, path_length = open_list.pop()
        for next_pos, length in waypoints[path[-1]]:
            if next_pos.imag == grid.height - 1:
                end_paths.append( (path + (next_pos,), path_length + length) )
            if next_pos in path:
                continue
            open_list.append((path + (next_pos,), path_length + length - 1))
    
    return sorted(end_paths, key=lambda n: n[1])[-1][1]
                
solve(data)

2282

In [20]:
solve(data, check_slopes=False)

6646